In [1]:
import pandas as pd
import pandasql as ps

Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.
Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах). В отношении Ships записаны название корабля, имя его класса и год спуска на воду. В отношение Battles включены название и дата битвы, в которой участвовали корабли, а в отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).
Замечания. 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 2) Потопленный корабль в последующих битвах участия не принимает.

![](scheme1.png)

In [2]:
classes = pd.read_csv('classes.csv', index_col='class')
ships = pd.read_csv('ships.csv', index_col='name')
outcomes = pd.read_csv('outcomes.csv', index_col=['ship', 'battle'])
battles = pd.read_csv('battles.csv', index_col='name')

Задача 1. Найдите страны, имевшие когда-либо классы обычных боевых кораблей ('bb') и имевшие когда-либо классы крейсеров ('bc').

![](scheme1.png)

In [18]:
query = '''
    select *
    from classes
    group by country
    having count(distinct type) = 2
'''
ps.sqldf(query)

,class,type,country,guns,bore,displacement
0,Renown,bc,Gt.Britain,6,15.0,32000
1,Kongo,bc,Japan,8,14.0,32000


In [4]:
# Использовать groupby и nunique
# Решить одной командой без использования merge и без дополнительных переменных
# Разрешается отдельно не выделять True страны

Задача 2.  Для каждого класса определите год, когда был спущен на воду первый корабль этого из класса.
![](scheme1.png)

In [21]:
query = '''
    select a.class, c
    from(
    select
        class
    from classes) a
    left join(
    select
        class, min(launched) as c
    from ships
    group by class) b
    on a.class = b.class
'''
ps.sqldf(query)

,class,c
0,Bismarck,NaN
1,Iowa,1943.0
2,Kongo,1913.0
3,North Carolina,1941.0
4,Renown,1916.0
5,Revenge,1916.0
6,Tennessee,1920.0
7,Yamato,1941.0


In [6]:
# Применить merge, groupby, min
# Решить одной командой без введения дополнительных переменных

Таблица company содержит идентификатор и название компании, осуществляющей перевозку пассажиров. Таблица trip содержит информацию о рейсах: номер рейса, идентификатор компании, тип самолета, город отправления, город прибытия, время отправления и время прибытия. Таблица passenger содержит идентификатор и имя пассажира. Таблица pass_in_trip содержит информацию о полетах: номер рейса, дата вылета (день), идентификатор пассажира и место, на котором он сидел во время полета.


![](scheme2.png)

In [26]:
trip = pd.read_csv('trip.csv', index_col='trip_no',
    parse_dates=['time_out', 'time_in'], infer_datetime_format=True)
pass_in_trip = pd.read_csv('pass_in_trip.csv',
    infer_datetime_format=True, parse_dates=['date'])
company = pd.read_csv('company.csv', index_col='id_comp')
passenger = pd.read_csv('passenger.csv', index_col='id_psg')

Задача 3. Определить имена пассажиров, когда-либо летевших на одном и том же месте более одного раза. 

![](scheme2.png)

In [42]:
query = '''
select name
from passenger
join (
    select *
    from pass_in_trip
    group by id_psg, place
    having count(place) > 1
) b
on passenger.id_psg = b.id_psg
'''
ps.sqldf(query)

,name
0,Bruce Willis
1,Nikole Kidman
2,Mullah Omar


In [9]:
# Решить одной командой используя один merge и два groupby, функции count и sum
# Запрещается введение дополнительных переменных

In [47]:
mask = pass_in_trip.groupby(['id_psg', 'place']).size() > 1
passenger.loc[mask[mask].index.get_level_values('id_psg')]

,name
id_psg,
1,Bruce Willis
8,Nikole Kidman
37,Mullah Omar


Задача 4.

Для каждой авиакомпании определить:

* количество выполненных перелетов;
* число использованных типов самолетов;
* количество перевезенных различных пассажиров;
* общее число перевезенных компанией пассажиров. 

![](scheme2.png)

In [71]:
query = '''
select *
from
    (select
        count(distinct trip_no),
        count(distinct plane),
        trip_no
    from trip
    group by id_comp) a
    join (select
        id_psg,
        count(id_psg),
        trip_no
    from pass_in_trip
    group by id_psg) as b
    on a.trip_no = b.trip_no
'''
ps.sqldf(query)

,count(distinct trip_no),count(distinct plane),trip_no,id_psg,count(id_psg),trip_no
0,6,2,1181,5,3,1181
1,6,2,1181,8,3,1181
2,2,1,1123,3,2,1123
3,2,1,1123,6,2,1123
4,2,1,1100,1,3,1100
5,10,1,7771,11,3,7771


In [78]:
query = '''
select count(id_psg), count(distinct id_psg), count(distinct plane), count(distinct a.trip_no)
from 
trip a left join pass_in_trip b
on a.trip_no = b.trip_no
group by id_comp
'''
ps.sqldf(query)

,count(id_psg),count(distinct id_psg),count(distinct plane),count(distinct a.trip_no)
0,9,6,2,6
1,2,2,1,2
2,4,4,1,2
3,1,1,1,2
4,16,6,1,10


In [11]:
# Решить одной командой используя nunique
# Запрещается введение дополнительных переменных

Задача 5. Найти количество маршрутов, которые обслуживаются наибольшим числом рейсов.
Замечания.
* A - B и B - A считать разными маршрутами.
* Использовать только таблицу Trip 
![](scheme2.png)

In [85]:
query = '''
select count(*) from (
        select count(distinct a.trip_no) as cnt
        from trip a
        group by a.town_from, a.town_to
    ) c
    where cnt = (
        select max(cnt) from 
        (
            select count(distinct a.trip_no) as cnt
            from trip a
            group by a.town_from, a.town_to
        )
)'''
ps.sqldf(query)

,count(*)
0,4


In [81]:
# Решить одной командой
# Не допускается ипользование merge и дополнительных переменных

Задача 6. Определить номера пассажиров, которые больше других времени провели в полетах.
Вывести имена и общее время в минутах, проведенное в полетах

![](scheme2.png)

In [14]:
# Решить одной командой без использования дополнительных переменных
# Здесь расматриваются типы времени: изначально timestamp, и после взятия разности -- timediff.
# Обращение к методам времени осуществляется через поле series.dt
# Использовать pd.DataFrame для создания промежуточного массива времен, groupby, sum и idxmax

14